
[LivePortrait GitHub](https://github.com/KwaiVGI/LivePortrait) <br>
[Hugging Face Model Card](https://huggingface.co/KwaiVGI/LivePortrait/tree/main)<br>
[Hugging Face Space](https://huggingface.co/spaces/KwaiVGI/LivePortrait)



In [ ]:
#@title Mount Google Drive for larger Video Input (It will create a folder Inside Google Drive called ```upload```. Then Upload your Video and Image in this folder.)
from IPython.display import clear_output
%cd /content/
from google.colab import drive
import os
import shutil
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/
!mkdir upload
clear_output()

In [ ]:
#@title Download Pretrained Weights and Install Packages
%cd /content/
!rm -rf ./LivePortrait
!git clone https://github.com/KwaiVGI/LivePortrait.git

from tqdm import tqdm
import urllib
import os
import shutil
def conditional_download(url, download_file_path):
    print(f"Downloading {os.path.basename(download_file_path)}")
    base_path = os.path.dirname(download_file_path)

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if os.path.exists(download_file_path):
        os.remove(download_file_path)

    try:
        request = urllib.request.urlopen(url)  # type: ignore[attr-defined]
        total = int(request.headers.get('Content-Length', 0))
    except urllib.error.URLError as e:
        print(f"Error: Unable to open the URL - {url}")
        print(f"Reason: {e.reason}")
        return

    with tqdm(total=total, desc='Downloading', unit='B', unit_scale=True, unit_divisor=1024) as progress:
        try:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))  # type: ignore[attr-defined]
        except urllib.error.URLError as e:
            print(f"Error: Failed to download the file from the URL - {url}")
            print(f"Reason: {e.reason}")
            return

    print(f"Download successful!")
    print(f"URL: {url}")
    print(f"Save at: {download_file_path}")




def download_models(base_path):
    # Function to download files
    def download_files(file_list, relative_path, base_url):
        for file_name in file_list:
            download_file_path = f"{base_path}/{relative_path}/{file_name}"
            conditional_download(f"{base_url}/{file_name}", download_file_path)

    # Define file lists and paths
    buffalo_l_files = ["2d106det.onnx", "det_10g.onnx"]
    buffalo_l_path = "LivePortrait/pretrained_weights/insightface/models/buffalo_l"
    buffalo_l_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/insightface/models/buffalo_l"

    base_models_files = [
        'appearance_feature_extractor.pth',
        'motion_extractor.pth',
        'spade_generator.pth',
        'warping_module.pth'
    ]
    base_models_path = "LivePortrait/pretrained_weights/liveportrait/base_models"
    base_models_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait/base_models"

    retargeting_models_files = ['stitching_retargeting_module.pth']
    retargeting_models_path = "LivePortrait/pretrained_weights/liveportrait/retargeting_models"
    retargeting_models_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait/retargeting_models"

    landmark_file = ["landmark.onnx"]
    landmark_path = "LivePortrait/pretrained_weights/liveportrait"
    landmark_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait"

    # Download files
    download_files(buffalo_l_files, buffalo_l_path, buffalo_l_url)
    download_files(base_models_files, base_models_path, base_models_url)
    download_files(retargeting_models_files, retargeting_models_path, retargeting_models_url)
    download_files(landmark_file, landmark_path, landmark_url)

#set up base_path
import os
root_path=os.getcwd()
# root_path="/content"
base_path=f"{root_path}"
#download model
download_models(base_path)
os.chdir(f"{base_path}/LivePortrait")
!pip install -r requirements.txt
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Utils
import os
if 'COLAB_GPU' in os.environ:
   base_path="/content"
os.chdir(f"{base_path}/LivePortrait")

import os
import subprocess
import re
import uuid
import shutil
def trim_video(input_file, save_folder, clip_duration=10, resolution="854x480"):
    # Ensure the save folder exists
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Get the total duration of the video in seconds
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", input_file],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT
    )
    total_duration = float(result.stdout)
    start_time = 0
    clip_number = 1

    while start_time < total_duration:
        output_file = os.path.join(save_folder, f"{clip_number}.mp4")
        command = (
            f"ffmpeg -vsync 0 -hwaccel cuda -c:v h264_cuvid -ss {start_time} -i {input_file} -t {clip_duration} "
            f"-vf scale={resolution} -c:v h264_nvenc -preset fast -b:v 5M -c:a copy {output_file} -y"
        )
        os.system(command)
        start_time += clip_duration
        clip_number += 1

def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else 0

def get_output_video(image_path,video_path):
  #extract base name only
  image_name=os.path.basename(image_path)
  video_name=os.path.basename(video_path)
  #only the filename without extention
  image_name=os.path.splitext(image_name)[0]
  video_name=os.path.splitext(video_name)[0]
  video_path=f"{image_name}--{video_name}.mp4"
  return video_path


def concatenate_videos(list_file, reference_video_path):
    video_name=os.path.basename(reference_video_path)
    video_name=os.path.splitext(video_name)[0]
    if not os.path.exists(f"{base_path}/final_video"):
      os.makedirs(f"{base_path}/final_video")
    random_uuid = uuid.uuid4()
    vid_name = f"{video_name}_LivePortrait_{str(random_uuid)[:6]}.mp4"
    output_path = f"{base_path}/final_video/{vid_name}"
    # Run FFmpeg to concatenate videos using the list.txt file
    ffmpeg_cmd=f"ffmpeg -f concat -safe 0 -i {list_file} -c copy {output_path} -y"
    var=os.system(ffmpeg_cmd)
    if var==0:
      print(f"Video saved at {output_path}")
      if os.path.exists("/content/gdrive/MyDrive/upload"):
        shutil.copy(output_path, f"/content/gdrive/MyDrive/upload/{vid_name}")
        print(f"Copied at drive '/content/gdrive/MyDrive/upload/{vid_name}'")
      return output_path
    else:
      print("error")
      return None


def make_clips(image_path,reference_video_path):
  if os.path.exists(f"{base_path}/generated_clips"):
    shutil.rmtree(f"{base_path}/generated_clips")
  if os.path.exists(f"{base_path}/input_video_clips"):
    shutil.rmtree(f"{base_path}/input_video_clips")
  if os.path.exists(f"{base_path}/video_clips"):
    shutil.rmtree(f"{base_path}/video_clips")
  os.makedirs(f"{base_path}/video_clips")
  save_folder = f"{base_path}/generated_clips"
  trim_video(reference_video_path, f"{base_path}/input_video_clips")
  file_list = os.listdir(f"{base_path}/input_video_clips")
  sorted_file_list = sorted(file_list, key=extract_number)
  for i in sorted_file_list:
    video_path=f"{base_path}/input_video_clips/{i}"
    if os.getcwd() != f"{base_path}/LivePortrait":
      os.chdir(f"{base_path}/LivePortrait")
    command=f"python inference.py -s {image_path} -d {video_path}  -o {save_folder} --flag_crop_driving_video"
    var=os.system(command)
    if var==0:
      print(f"{base_path}/generated_clips/{i}")
      result_video=get_output_video(image_path,video_path)
      shutil.copy(f"{base_path}/generated_clips/{result_video}",f"{base_path}/video_clips/{i}")
      print(f"copied {base_path}/generated_clips/{result_video}")
    else:
      print(f"error in {i}")
  sorted_video_list = sorted(os.listdir(f"{base_path}/video_clips"), key=extract_number)
  with open(f"{base_path}/join.txt", 'w') as f:
    for i in sorted_video_list:
      f.write(f"file '{base_path}/video_clips/{i}'\n")
  final_video_name=concatenate_videos(f"{base_path}/join.txt", reference_video_path)
  return final_video_name



In [ ]:
# @title ##**Choose option** { display-mode: "form" }
choose= "already_uploaded"#@param ['upload_now','already_uploaded']
import os

folder_path = "/content/gdrive/MyDrive/upload"

# Check if the folder exists
if not os.path.exists(folder_path):
    # If it doesn't exist, create it
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

%cd /content/gdrive/MyDrive/upload
from IPython.display import clear_output
clear_output()
f_names=[]
from google.colab import files
if choose== "upload_now":
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(fn)
  clear_output()

video_id=1
table={}
ids=[]
videos=[]
id_monitor={}
video_folder="/content/gdrive/MyDrive/upload"
for i in os.listdir(video_folder):
  ids.append(video_id)
  videos.append(i)
  id_monitor[video_id]=i
  video_id=video_id+1
table["file_name"]=videos
table["file_id"]=ids
import pandas as pd
df = pd.DataFrame(table)
try:
  df.set_index('file_id', inplace=True)
except:
  pass
print(df)
print("\n")


               file_name
file_id                 
1        jio_ALTERED.mp4
2              king.jpeg




In [ ]:
# @title Enter input Image ID
image_path=""
if choose== "already_uploaded":
  image_id =   1#@param {type: "number"}
  input_image_path="/content/gdrive/MyDrive/upload/"+id_monitor[int(image_id)]
input_image_path

'/content/gdrive/MyDrive/upload/jio_ALTERED.mp4'

In [ ]:
# @title Enter input video ID
video_path=""
if choose== "already_uploaded":
  video_id =   2#@param {type: "number"}
  input_video_path="/content/gdrive/MyDrive/upload/"+id_monitor[int(video_id)]
input_video_path

'/content/gdrive/MyDrive/upload/king.jpeg'

In [ ]:
Image_Path= '/content/gdrive/MyDrive/upload/king.jpeg'  # @param {type: "string"}
Reference_Video_Path = '/content/gdrive/MyDrive/upload/jio_ALTERED.mp4'  # @param {type: "string"}

save_video_path=make_clips(Image_Path,Reference_Video_Path)
from IPython.display import clear_output
clear_output()
from google.colab import files
print(f"File save at {save_video_path}")
files.download(save_video_path)

/content/generated_clips/1.mp4
copied /content/generated_clips/king--1.mp4
/content/generated_clips/2.mp4
copied /content/generated_clips/king--2.mp4
/content/generated_clips/3.mp4
copied /content/generated_clips/king--3.mp4
error in 4.mp4
error in 5.mp4
error in 6.mp4
error in 7.mp4
error in 8.mp4
/content/generated_clips/9.mp4
copied /content/generated_clips/king--9.mp4
/content/generated_clips/10.mp4
copied /content/generated_clips/king--10.mp4
/content/generated_clips/11.mp4
copied /content/generated_clips/king--11.mp4
/content/generated_clips/12.mp4
copied /content/generated_clips/king--12.mp4
/content/generated_clips/13.mp4
copied /content/generated_clips/king--13.mp4
/content/generated_clips/14.mp4
copied /content/generated_clips/king--14.mp4
/content/generated_clips/15.mp4
copied /content/generated_clips/king--15.mp4
/content/generated_clips/16.mp4
copied /content/generated_clips/king--16.mp4
/content/generated_clips/17.mp4
copied /content/generated_clips/king--17.mp4
/content/